In [1]:
import os
print('Hello world')
# get current path:
os.getcwd()

Hello world


'/home/tomas/Projects/visp_matlab_loader/tests'

In [2]:
import os
# Use os to add path '..' to sys.path
os.sys.path.append('..')
import visp_matlab_loader as vml





In [3]:
#  Find installed MATLAB versions

# from visp_matlab_loader.matlab_compiler import MatlabVersionFinder, MatlabCompiler
from visp_matlab_loader.matlab_path_setter import MatlabPathSetter
matlab_path = MatlabPathSetter()
print(matlab_path.matlab_root)
if not matlab_path.verify_paths():
    raise Exception('Matlab paths are not set correctly')

# MatlabCompiler(finder).compile()


/usr/local/MATLAB/R2023b


In [4]:
from visp_matlab_loader import matlab_compiler
# reload matlab_compiler
import importlib
importlib.reload(matlab_compiler)
from visp_matlab_loader import matlab_compiler



MATLAB_LIBRARY_PATH = os.path.join(os.getcwd(), '..', 'matlab', 'libraries')
MATLAB_COMPILED_PATH = os.path.join(os.getcwd(), '..', 'matlab', 'compiled')

# Verify pat exists
if not os.path.exists(MATLAB_LIBRARY_PATH):
    raise Exception('Matlab library path does not exist: {}'.format(MATLAB_LIBRARY_PATH))

if not os.path.exists(MATLAB_COMPILED_PATH):    
    os.makedirs(MATLAB_COMPILED_PATH)

for name in os.listdir(MATLAB_LIBRARY_PATH):
    if os.path.isdir(os.path.join(MATLAB_LIBRARY_PATH, name)):
        print('Compiling {}'.format(name))
        compiler = matlab_compiler.MATLABProjectCompiler(project_path=os.path.join(MATLAB_LIBRARY_PATH, name), 
                                         output_path=os.path.join(MATLAB_COMPILED_PATH, name)) # Here we would add a MatlabPathSetter if we want to specify which compiler to use!
        compiler_code, compiler_message = compiler.compile_project(verbose=False)
        if compiler_code != 0:
            print(f'Error compiling project: {name}, compiler message was: {compiler_message}')
        else:
            print(f'Successfully compiled project: {name}')
        
                                 

Compiling get_next_thousand
Error compiling project: get_next_thousand, compiler message was: Output file get_next_thousand already exists in /home/tomas/Projects/visp_matlab_loader/matlab/compiled/get_next_thousand.
Compiling voice_analysis_toolbox
Successfully compiled project: voice_analysis_toolbox
Compiling covarep
Error compiling project: covarep, compiler message was: Output file covarep already exists in /home/tomas/Projects/visp_matlab_loader/matlab/compiled/covarep.


In [5]:
# Find compiled projects
from visp_matlab_loader import find_compiled_projects
importlib.reload(find_compiled_projects)
from visp_matlab_loader import find_compiled_projects

compiled_projects = find_compiled_projects.CompiledProjectFinder('../matlab/compiled/')
for project in compiled_projects.compiled_projects:
    print(project)




Compiled files for project 'get_next_thousand:' 
	Located at '/home/tomas/Projects/visp_matlab_loader/matlab/compiled/get_next_thousand'.
 	Wrapper file is '/home/tomas/Projects/visp_matlab_loader/matlab/compiled/get_next_thousand/get_next_thousand_wrapper.m', 
	Binary file is '/home/tomas/Projects/visp_matlab_loader/matlab/compiled/get_next_thousand/get_next_thousand'.
 	Code found at '/home/tomas/Projects/visp_matlab_loader/matlab/libraries/get_next_thousand'.
	Test cases are located at '/home/tomas/Projects/visp_matlab_loader/matlab/tests/get_next_thousand'.
Compiled files for project 'voice_analysis_toolbox:' 
	Located at '/home/tomas/Projects/visp_matlab_loader/matlab/compiled/voice_analysis_toolbox'.
 	Wrapper file is '/home/tomas/Projects/visp_matlab_loader/matlab/compiled/voice_analysis_toolbox/voice_analysis_toolbox_wrapper.m', 
	Binary file is '/home/tomas/Projects/visp_matlab_loader/matlab/compiled/voice_analysis_toolbox/voice_analysis_toolbox'.
 	Code found at '/home/tomas/

In [6]:
# Now we have the compiled projects, and can check if any of them have any test data

TEST_CASE_DIRECTORY = os.path.join(os.getcwd(), '..', 'tests', 'library_tests')

for project in compiled_projects.compiled_projects:
    name = project.name
    test_dir = project.test_case_directory
    print(f'Project {name}, test directory: ', project.test_case_directory)
    # Check if it exists:
    if project.test_case_files:
        print('Test case files:')
        for test_case in project.test_case_files:
            print(test_case)
            

Project get_next_thousand, test directory:  /home/tomas/Projects/visp_matlab_loader/matlab/tests/get_next_thousand
Test case files:
/home/tomas/Projects/visp_matlab_loader/matlab/tests/get_next_thousand/simple_test.json
Project voice_analysis_toolbox, test directory:  None
Project covarep, test directory:  /home/tomas/Projects/visp_matlab_loader/matlab/tests/covarep


In [7]:
import visp_matlab_loader.compiled_project_executor
importlib.reload(visp_matlab_loader.compiled_project_executor)
from visp_matlab_loader.compiled_project_executor import ExecuteCompiledProject

executor = None

for project in compiled_projects.compiled_projects:
    print(project)
    executor = ExecuteCompiledProject(project,return_inputs=True)
    print(executor.available_functions)
    # executor.execute_script('')
    break

if executor:
    # executor.available_functions['getnextthousand']
    result = executor.execute_script("getnextthousand", 1, 1)

Compiled files for project 'get_next_thousand:' 
	Located at '/home/tomas/Projects/visp_matlab_loader/matlab/compiled/get_next_thousand'.
 	Wrapper file is '/home/tomas/Projects/visp_matlab_loader/matlab/compiled/get_next_thousand/get_next_thousand_wrapper.m', 
	Binary file is '/home/tomas/Projects/visp_matlab_loader/matlab/compiled/get_next_thousand/get_next_thousand'.
 	Code found at '/home/tomas/Projects/visp_matlab_loader/matlab/libraries/get_next_thousand'.
	Test cases are located at '/home/tomas/Projects/visp_matlab_loader/matlab/tests/get_next_thousand'.
{'getnextthousand': {'output': ['nextNumbers'], 'input': ['inputNumber']}, 'getnexttwothousand': {'output': ['nextNumbers'], 'input': ['inputNumber']}}
[1]


In [8]:
from visp_matlab_loader.matlab_execution_result import MatlabExecutionResult


test_json = result.to_json()
loaded_results = MatlabExecutionResult.from_json(test_json)
loaded_results.inputs

executor = ExecuteCompiledProject(project,return_inputs=True)
new_result = executor.execute_script(loaded_results.function_name, len(loaded_results.outputs), *loaded_results.inputs)
print(new_result == loaded_results)


[1.0]
True


In [20]:
# Try to run the modified voice analysis toolbox script
# executor = ExecuteCompiledProject(],return_inputs=True)
va_toolbox = compiled_projects.get_project('voice_analysis_toolbox')
va_executor = ExecuteCompiledProject(va_toolbox,return_inputs=True)


from thefuzz import fuzz
# Create function that finds functions containing strings similar to (not necessarily identical) to the given string
def find_similar_functions(string, executor, ratio_threshold=50):
    # Get all functions
    all_functions = executor.available_functions
    # Find all functions that contain the given string
    similar_functions = []
    
    for function in all_functions:
        if string in function:
            similar_functions.append(function)
            continue
        # Use fuzzy string matching to find similar strings, using the thefuzz library
        
        ratio = fuzz.partial_ratio(string, function)
        if ratio > ratio_threshold:
            similar_functions.append(function)
        

    return similar_functions

# try finding a project containing the string 'modified'
find_similar_functions('modified', va_executor, ratio_threshold=100)


['voice_analysis_modified']

In [9]:
# Run test cases:
from visp_matlab_loader import test_case_runner
importlib.reload(test_case_runner)
from visp_matlab_loader.test_case_runner import TestCaseRunner
for project in compiled_projects.compiled_projects:
    test_dir = project.test_case_directory
    print('EXPECTED TEST DIRECTORY: ', test_dir)
    if not project.test_case_files:
        print('Skipping project: {}, no test cases'.format(project.name))
        continue
    
    test_case_runner = TestCaseRunner(project, test_case)
    for test in test_case:
        print(test)
        


EXPECTED TEST DIRECTORY:  /home/tomas/Projects/visp_matlab_loader/matlab/tests/get_next_thousand


NameError: name 'json_numpy' is not defined

In [ ]:

import visp_matlab_loader
importlib.reload(visp_matlab_loader.compiled_project_executor)
from visp_matlab_loader.compiled_project_executor import ExecuteCompiledProject



for project in compiled_projects.compiled_projects:
    print(project)
    executor = ExecuteCompiledProject(project)
    print(executor.available_functions)
    # executor.execute_script('')
    break

# executor.available_functions['getnextthousand']
result = executor.execute_script("getnextthousand", 1, 1)

ModuleNotFoundError: No module named 'visp_matlab_loader'

In [ ]:
!cd ..
!ls

Compile_And_Test_Projects.ipynb  output.wav
Test_Voice_Analysis.ipynb	 output2.wav
input.mat			 results.mat
library_tests			 test_script.py
logs				 voice_analysis_directory_output.mat
output


In [ ]:
va_toolbox = compiled_projects.get_project('voice_analysis_toolbox')
print(va_toolbox)
executor = ExecuteCompiledProject(va_toolbox)
executor.available_functions

executor.find_available_function('analysis_modified')
executor.execute_script('voice_analysis_modified', )

Compiled files for project 'voice_analysis_toolbox:' 
	Located at '/home/tomas/Projects/visp_matlab_loader/matlab/compiled/voice_analysis_toolbox'.
 	Wrapper file is '/home/tomas/Projects/visp_matlab_loader/matlab/compiled/voice_analysis_toolbox/voice_analysis_toolbox_wrapper.m', 
	Binary file is '/home/tomas/Projects/visp_matlab_loader/matlab/compiled/voice_analysis_toolbox/voice_analysis_toolbox'.
 	Code found at '/home/tomas/Projects/visp_matlab_loader/matlab/libraries/voice_analysis_toolbox'.
	Test cases are located at 'None'.


TypeError: ExecuteCompiledProject.execute_script() missing 1 required positional argument: 'output_count'